# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:

    --> Name: Trilok Sachin Chittala
    --> BITS ID: 2022AC050732
    --> Group Number: 159
    --> Section: 1


## Problem: Rabbit Rescue Assignment

### 1.	Environment and Agent Description

PEAS Description: Rabbit Rescue Agent

Performance Measure: The performance measure for the Rabbit Rescue Agent is the successful and efficient rescue of the trapped rabbit. The agent aims to find the optimal path from the rabbit's current position to the safety of the goal position while minimizing the total cost of the path. The cost includes a fixed value of +3 for each cell transition. Additionally, the agent must consider penalties of +5 cost for moving towards a fire (F) cell, indicating a hazardous area, and +1 cost for moving towards a bush (B) cell, representing a potentially obstructed region. The optimal path is one that achieves the rescue mission while avoiding dangerous areas, navigating around obstacles like walls (W), and ensuring the rabbit's safety.

Environment: The environment where the Rabbit Rescue Agent operates is represented as a grid. This grid contains various cell types, including walls (W) that restrict the agent's movement. The presence of fires (F) indicates hazardous zones, and moving towards these cells incurs a significant penalty. Bushes (B) indicate potentially obstructed areas, which also incur a smaller penalty. The agent's goal is to navigate through this dynamic and challenging environment to reach the trapped rabbit, guiding it safely to the goal cell.

Actuator: The actuator in the Rabbit Rescue Agent is responsible for executing movements within the grid environment. It enables the agent to move in four directions: north, south, west, and east. By controlling the actuator, the agent can traverse the grid, explore different paths, and ultimately find the optimal route for rescuing the trapped rabbit while avoiding hazardous cells and minimizing the total cost.

Sensor: The agent is equipped with sensors to perceive its surroundings and gather vital information for decision-making during the rescue mission. These sensors allow the agent to detect its current position within the grid, identify the cell types at its location (fire, bush, wall, or open cell), and calculate the Euclidean distance to the goal position using a heuristic function. The sensors provide essential feedback to guide the agent's pathfinding process, enabling it to make informed decisions about the safest and most efficient route to the goal.

The Rabbit Rescue Agent's intelligent navigation, guided by its sensors and actuator, ensures that it can successfully navigate through the complex grid environment, avoid hazardous areas, and reach the goal position to rescue the trapped rabbit effectively. The agent's performance is measured by its ability to find the optimal path, ensuring the rabbit's safety, and minimizing the total cost of the rescue mission.

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [2]:
#Importing Libraries
%load_ext memory_profiler
import heapq

import random
import math

import time
import resource
from memory_profiler import profile

### DYNAMIC INPUT

In [14]:
#Code Block : Set Initial State (Must handle dynamic inputs)

#Initial State of the Rabbit
start = (0, 2)
goal = (10, 7)

#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

#Rabbit MAZE
grid = [
    ['B', '.', '.', '.', '.', 'W', '.', '.', '.', 'F'],
    ['.', 'W', '.', '.', '.', 'W', 'W', 'W', 'W', '.'],
    ['.', 'W', '.', '.', '.', 'W', '.', 'B', 'W', '.'],
    ['.', 'W', 'W', '.', '.', 'W', 'W', '.', '.', '.'],
    ['.', 'F', 'W', '.', '.', 'W', 'F', '.', '.', '.'],
    ['.', 'W', 'W', '.', '.', 'W', 'W', '.', 'W', 'W'],
    ['.', 'W', 'B', '.', '.', 'W', '.', '.', '.', 'B'],
    ['W', 'W', '.', 'W', 'W', 'W', 'W', '.', 'W', '.'],
    ['.', '.', '.', '.', '.', '.', 'W', '.', 'W', '.'],
    ['.', '.', '.', 'W', 'W', '.', '.', '.', 'W', '.'],
    ['.', '.', '.', 'W', '.', '.', '.', '.', 'W', '.']
]


#Action Space of the Rabbit
movements = [(-1, 0), (1, 0), (0, -1), (0, 1)]

#Defining Cost Values
ACTION_COST = 3 #Cost for taking an action
BUSH_COST = 1 #Additional cost for Bush
FIRE_COST = 3 #Additional cost for Fire

In [15]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def heuristic(position):
    # Calculate Euclidean distance from the current position to the goal position
    return math.sqrt((position[0] - goal[0]) ** 2 + (position[1] - goal[1]) ** 2)

### 2.	Definition of Algorithm 1 (A* Search)

In [16]:
#Code Block : Function for algorithm 1 implementation
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

#Function block for A star search
def a_star(grid, start, goal):
    
    open_list = []
    closed_set = set()
    heapq.heappush(open_list, (0, start))  # Priority queue with total cost and position

    # Initialize dictionaries to store cost and parent information
    cost_so_far = {start: 0}
    parent = {start: None}

    while open_list:
        _, current = heapq.heappop(open_list)
        

        if current == goal:
            # Path found, trace back the path
            path = []
            while current is not None:
                path.append(current)
                current = parent[current]
            path.reverse()

            updated_grid = grid.copy()
            for position in path:
                updated_grid[position[0]][position[1]] = 'P'
                
            return path, updated_grid, cost_so_far

        closed_set.add(current)

        for move in movements:
            new_position = (current[0] + move[0], current[1] + move[1])

            if 0 <= new_position[0] < len(grid) and 0 <= new_position[1] < len(grid[0]):
                if grid[new_position[0]][new_position[1]] == 'W' or new_position in closed_set: 
                    
                    continue

                if grid[new_position[0]][new_position[1]] == 'F':
                    new_cost = cost_so_far[current] + FIRE_COST + ACTION_COST
                elif grid[new_position[0]][new_position[1]] == 'B':
                    new_cost = cost_so_far[current] + BUSH_COST + ACTION_COST
                else:
                    new_cost = cost_so_far[current] + ACTION_COST

                if new_position not in cost_so_far or new_cost < cost_so_far[new_position]:
                    cost_so_far[new_position] = new_cost
                    total_cost = new_cost + heuristic(new_position)
                    heapq.heappush(open_list, (total_cost, new_position))
                    parent[new_position] = current

    return None  # No path found



### 3.	Definition of Algorithm 2 (Random Restart Hill Climbing Algorithm)

In [17]:
#Code Block : Function for algorithm 2 implementation

##Function Block for Random Restart Hill Climbing

def random_restart_hill_climbing(grid, start, goal, max_iterations=100):
    best_path = None
    best_cost = float('inf')

    current = start
    path = [current]
    cost = 0

    closed_set = set()
    updated_grid = grid.copy()

    for i in range(max_iterations):
        
        j = 0
        while current != goal:

            closed_set.add(current)
            next_moves = []
            for move in movements:
                new_position = (current[0] + move[0], current[1] + move[1])

                if 0 <= new_position[0] < len(grid) and 0 <= new_position[1] < len(grid[0]):
                    if grid[new_position[0]][new_position[1]] == 'W' or new_position in closed_set:
                        
                        continue

                    if grid[new_position[0]][new_position[1]] == 'F':
                        new_cost = cost + FIRE_COST + ACTION_COST
                        total_cost = new_cost + heuristic(new_position)
                        
                    elif grid[new_position[0]][new_position[1]] == 'B':
                        new_cost = cost + ACTION_COST + BUSH_COST
                        total_cost = new_cost + heuristic(new_position)
                        
                    else:
                        new_cost = cost + ACTION_COST
                        total_cost = new_cost + heuristic(new_position)
                        

                    next_moves.append((new_position, total_cost))
            


            # print(i, current, next_moves)
            if not next_moves:
                # No valid moves from the current position, restart
                print(i, next_moves, 'Agent Stuck, No valid moves from the current position, restarting')
                break

        

            next_moves.sort(key=lambda x: x[1])  # Sort moves by cost
            current, cost = next_moves[0]  # Move to the best neighbor
            path.append(current)

            updated_grid[current[0]][current[1]] = 'P'

            

            if current == goal and cost < best_cost:
                # Found a better path
                best_path = path
                best_cost = cost
                break
            
    return best_path, updated_grid, best_cost

### 4.	Calling the search algorithms


In [18]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
# Find the optimal path Using A Star
results = a_star(grid, start, goal)

print('A* Search Algorithm Results\n')
if results:
    path, upd_grid, cost_so_far = results
    print("Optimal Path:")
    for position in path:
        print('Position:', position, 'Cost->', cost_so_far[position])

    print("Updated Grid:")
    display(upd_grid)

    print("Path Cost:", cost_so_far[goal])
else:
    print("No path found.")

A* Search Algorithm Results

Optimal Path:
Position: (0, 2) Cost-> 0
Position: (1, 2) Cost-> 3
Position: (2, 2) Cost-> 6
Position: (2, 3) Cost-> 9
Position: (3, 3) Cost-> 12
Position: (4, 3) Cost-> 15
Position: (5, 3) Cost-> 18
Position: (6, 3) Cost-> 21
Position: (6, 2) Cost-> 25
Position: (7, 2) Cost-> 28
Position: (8, 2) Cost-> 31
Position: (8, 3) Cost-> 34
Position: (8, 4) Cost-> 37
Position: (8, 5) Cost-> 40
Position: (9, 5) Cost-> 43
Position: (9, 6) Cost-> 46
Position: (9, 7) Cost-> 49
Position: (10, 7) Cost-> 52
Updated Grid:


[['B', '.', 'P', '.', '.', 'W', '.', '.', '.', 'F'],
 ['.', 'W', 'P', '.', '.', 'W', 'W', 'W', 'W', '.'],
 ['.', 'W', 'P', 'P', '.', 'W', '.', 'B', 'W', '.'],
 ['.', 'W', 'W', 'P', '.', 'W', 'W', '.', '.', '.'],
 ['.', 'F', 'W', 'P', '.', 'W', 'F', '.', '.', '.'],
 ['.', 'W', 'W', 'P', '.', 'W', 'W', '.', 'W', 'W'],
 ['.', 'W', 'P', 'P', '.', 'W', '.', '.', '.', 'B'],
 ['W', 'W', 'P', 'W', 'W', 'W', 'W', '.', 'W', '.'],
 ['.', '.', 'P', 'P', 'P', 'P', 'W', '.', 'W', '.'],
 ['.', '.', '.', 'W', 'W', 'P', 'P', 'P', 'W', '.'],
 ['.', '.', '.', 'W', '.', '.', '.', 'P', 'W', '.']]

Path Cost: 52


In [8]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
# Find the optimal path using Random Restart Hill Climbing

path, updated_grid, best_cost = random_restart_hill_climbing(grid, start, goal)

print('Random Restart Hill Climbing Algorithm Results\n')
if path:
    print("Optimal Path:")
    for position in path:
        print('Position:', position, 'Cost->', cost_so_far[position])

    print("Updated Grid:")
    display(updated_grid)

else:
    print("No path found.")

Random Restart Hill Climbing Algorithm Results

Optimal Path:
Position: (0, 2) Cost-> 0
Position: (1, 2) Cost-> 3
Position: (2, 2) Cost-> 6
Position: (2, 3) Cost-> 9
Position: (3, 3) Cost-> 12
Position: (4, 3) Cost-> 15
Position: (5, 3) Cost-> 18
Position: (6, 3) Cost-> 21
Position: (6, 4) Cost-> 24
Position: (5, 4) Cost-> 21
Position: (4, 4) Cost-> 18
Position: (3, 4) Cost-> 15
Position: (2, 4) Cost-> 12
Position: (2, 5) Cost-> 15
Position: (2, 6) Cost-> 18
Position: (2, 7) Cost-> 22
Position: (3, 7) Cost-> 25
Position: (4, 7) Cost-> 28
Position: (5, 7) Cost-> 31
Position: (6, 7) Cost-> 34
Position: (7, 7) Cost-> 37
Position: (8, 7) Cost-> 40
Position: (9, 7) Cost-> 43
Position: (10, 7) Cost-> 46
Updated Grid:


[['B', '.', 'P', '.', '.', 'W', '.', '.', '.', 'F'],
 ['.', 'W', 'P', '.', '.', 'W', 'W', 'W', 'W', '.'],
 ['.', 'W', 'P', 'P', 'P', 'P', 'P', 'P', 'W', '.'],
 ['.', 'W', 'W', 'P', 'P', 'W', 'W', 'P', '.', '.'],
 ['.', 'F', 'W', 'P', 'P', 'W', 'F', 'P', '.', '.'],
 ['.', 'W', 'W', 'P', 'P', 'W', 'W', 'P', 'W', 'W'],
 ['.', 'W', 'B', 'P', 'P', 'W', '.', 'P', '.', 'B'],
 ['W', 'W', '.', 'W', 'W', 'W', 'W', 'P', 'W', '.'],
 ['.', '.', '.', '.', '.', '.', 'W', 'P', 'W', '.'],
 ['.', '.', '.', 'W', 'W', '.', '.', 'P', 'W', '.'],
 ['.', '.', '.', 'W', '.', '.', '.', 'P', 'W', '.']]

### 5.	Comparitive Analysis

In [17]:
# Print the time and space complexity of the algorithms
print("Time and Space Complexity for A* Algorithm:")
print('Space Utilized:')

%memit a_star(grid, start, goal)

# Time complexity measurement for A* algorithm
start_time = time.time()
a_star_path = a_star(grid, start, goal)
end_time = time.time()
a_star_time = end_time - start_time


print(f"Time Complexity: {a_star_time:.6f} seconds")



Time and Space Complexity for A* Algorithm:
Space Utilized:
peak memory: 63.49 MiB, increment: 1.90 MiB
Time Complexity: 0.000282 seconds


In [18]:
#Code Block : Print the Time & Space complexity of algorithm 2
# Time and space complexity measurement for Random Restart Hill Climbing algorithm

print("Time and Space Complexity for Random Restart Hill Climbing Algorithm:")
print('Space Utilized:')
%memit rrhc_path = random_restart_hill_climbing(grid, start, goal, max_iterations=1000)

start_time = time.time()
rrhc_path = random_restart_hill_climbing(grid, start, goal, max_iterations=1000)
end_time = time.time()
rrhc_time = end_time - start_time

print(f"Time Complexity: {rrhc_time:.6f} seconds")


Time and Space Complexity for Random Restart Hill Climbing Algorithm:
Space Utilized:
peak memory: 62.98 MiB, increment: -0.79 MiB
Time Complexity: 0.000176 seconds


### Comparison :

--The A* algorithm has a more predictable time complexity (in terms of search depth) and often performs efficiently for well-designed heuristic functions.

--The Random Restart Hill Climbing algorithm does not have a precise time complexity but can benefit from random restarts to explore different paths and escape local optima.

--The A* algorithm usually finds the optimal path if the heuristic is admissible and consistent, but it may require more time and memory for large search spaces.

--The Random Restart Hill Climbing algorithm can provide approximate solutions efficiently, but the quality of the result can vary based on the number of restarts and the path evaluation function.

--In terms of space complexity, A* requires more memory due to the need to maintain the open and closed lists, while the Random Restart Hill Climbing algorithm generally consumes less memory.

--Overall, the A* algorithm is suitable when finding the optimal path is crucial, and sufficient memory is available. On the other hand, the Random Restart Hill Climbing algorithm is suitable when fast execution and approximate solutions are acceptable, especially in scenarios with limited memory resources.